In [1]:
import sys
import os

# Get the current working directory
current_dir = os.getcwd()

# Add the parent directory to sys.path
parent_dir = os.path.abspath(os.path.join(current_dir, '..'))
sys.path.insert(0, parent_dir)

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.ticker import ScalarFormatter
from urllib.request import urlretrieve
from scipy.stats import pearsonr

from config.config import DATA_PATH


In [3]:
pd.set_option('display.max_columns', None)

In [21]:
flights = pd.read_csv(DATA_PATH + '/interim/2023-performance-data-clean.csv',  parse_dates=['scheduled_departure_datetime'])

In [22]:
flights.head()

,year,quarter,month,day_of_month,day_of_week,marketing_airline_id,flight_number_marketing_airline,operating_airline_id,tail_number,origin_airport_id,origin_city_market_id,origin,origin_state,dest_airport_id,dest_city_market_id,dest,dest_state,dep_delay,dep_del15,departure_delay_groups,taxi_out,taxi_in,arr_delay,arr_del15,arrival_delay_groups,cancelled,diverted,scheduled_elapsed_time,actual_elapsed_time,air_time,distance,distance_group,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay,div_airport_landings,code_share_flight,origin_city,destination_city,scheduled_departure_datetime,scheduled_arrival_datetime,actual_departure_datetime,actual_arrival_datetime,overnight_depature,overnight_arrival,airline_mkt,airline_ops,origin_timezone,destination_timezone,scheduled_departure_datetime_utc,actual_departure_datetime_utc,scheduled_arrival_datetime_utc,actual_arrival_datetime_utc,is_holiday
0,2023,3,8,6,7,20416,2252,20416,N978NK,12889,32211,LAS,NV,11057,31057,CLT,NC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,263.0,NaN,NaN,1916.0,8,NaN,NaN,NaN,NaN,NaN,0.0,0,Las Vegas,Charlotte,2023-08-06 23:59:00,2023-08-07 07:22:00,NaN,NaN,0,0,Spirit Air Lines,Spirit Air Lines,America/Los_Angeles,America/New_York,2023-08-07 07:00:00+00:00,NaN,2023-08-07 11:00:00+00:00,NaN,0
1,2023,3,8,7,1,20416,2252,20416,N974NK,12889,32211,LAS,NV,11057,31057,CLT,NC,76.0,1.0,5.0,19.0,9.0,69.0,1.0,4.0,0,0,263.0,256.0,228.0,1916.0,8,3.0,0.0,1.0,0.0,65.0,0.0,0,Las Vegas,Charlotte,2023-08-07 23:59:00,2023-08-08 07:22:00,2023-08-08 01:15:00,2023-08-08 08:31:00,0,0,Spirit Air Lines,Spirit Air Lines,America/Los_Angeles,America/New_York,2023-08-08 07:00:00+00:00,2023-08-08 08:15:00+00:00,2023-08-08 11:00:00+00:00,2023-08-08 12:31:00+00:00,0
2,2023,3,8,9,3,20416,2252,20416,N519NK,12889,32211,LAS,NV,11057,31057,CLT,NC,-11.0,0.0,-1.0,14.0,10.0,-13.0,0.0,-1.0,0,0,258.0,256.0,232.0,1916.0,8,NaN,NaN,NaN,NaN,NaN,0.0,0,Las Vegas,Charlotte,2023-08-09 23:10:00,2023-08-10 06:28:00,2023-08-09 22:59:00,2023-08-10 06:15:00,0,0,Spirit Air Lines,Spirit Air Lines,America/Los_Angeles,America/New_York,2023-08-10 06:00:00+00:00,2023-08-10 05:59:00+00:00,2023-08-10 10:00:00+00:00,2023-08-10 10:15:00+00:00,0
3,2023,3,8,10,4,20416,2252,20416,N532NK,12889,32211,LAS,NV,11057,31057,CLT,NC,-8.0,0.0,-1.0,12.0,7.0,-30.0,0.0,-2.0,0,0,258.0,236.0,217.0,1916.0,8,NaN,NaN,NaN,NaN,NaN,0.0,0,Las Vegas,Charlotte,2023-08-10 23:10:00,2023-08-11 06:28:00,2023-08-10 23:02:00,2023-08-11 05:58:00,0,0,Spirit Air Lines,Spirit Air Lines,America/Los_Angeles,America/New_York,2023-08-11 06:00:00+00:00,2023-08-11 06:02:00+00:00,2023-08-11 10:00:00+00:00,2023-08-11 09:58:00+00:00,0
4,2023,3,8,12,6,20416,2252,20416,N529NK,12889,32211,LAS,NV,11057,31057,CLT,NC,4.0,0.0,0.0,18.0,6.0,-4.0,0.0,-1.0,0,0,258.0,250.0,226.0,1916.0,8,NaN,NaN,NaN,NaN,NaN,0.0,0,Las Vegas,Charlotte,2023-08-12 23:10:00,2023-08-13 06:28:00,2023-08-12 23:14:00,2023-08-13 06:24:00,0,0,Spirit Air Lines,Spirit Air Lines,America/Los_Angeles,America/New_York,2023-08-13 06:00:00+00:00,2023-08-13 06:14:00+00:00,2023-08-13 10:00:00+00:00,2023-08-13 10:24:00+00:00,0


In [23]:
flights.shape

(7260941, 56)

In [24]:
flights.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7260941 entries, 0 to 7260940
Data columns (total 56 columns):
 #   Column                            Dtype         
---  ------                            -----         
 0   year                              int64         
 1   quarter                           int64         
 2   month                             int64         
 3   day_of_month                      int64         
 4   day_of_week                       int64         
 5   marketing_airline_id              int64         
 6   flight_number_marketing_airline   int64         
 7   operating_airline_id              int64         
 8   tail_number                       object        
 9   origin_airport_id                 int64         
 10  origin_city_market_id             int64         
 11  origin                            object        
 12  origin_state                      object        
 13  dest_airport_id                   int64         
 14  dest_city_market_i

In [32]:
routes = flights.groupby(['origin', 'dest'])[['origin', 'dest']].size().reset_index(name='flight_count')
routes.index += 1
routes = routes.reset_index().rename(columns={'index':'route_id'})
routes

,route_id,origin,dest,flight_count
0,1,ABE,ATL,992
1,2,ABE,BNA,139
2,3,ABE,CLT,1092
3,4,ABE,DEN,58
4,5,ABE,FLL,95
...,...,...,...,...
6709,6710,YAK,CDV,355
6710,6711,YAK,JNU,359
6711,6712,YKM,SEA,411
6712,6713,YUM,DFW,449


In [34]:
route_mapping = routes.set_index(['origin', 'dest'])['route_id'].to_dict()

In [35]:
flights['route_id'] = flights[['origin', 'dest']].apply(tuple, axis=1).map(route_mapping)

12-4: overnight flight
4-6: early morning 
6-11: morning
11-1: midday
1-3: early afternoon
3-5: afternoon
5-7: evening
7 - 10: night
10-12: late night

In [36]:
bins = [0, 4, 6, 11, 13, 15, 17, 19, 22, 24]
labels = ['overnight', 'early morning', 'morning', 'midday', 'early afternoon', 'afternoon', 'evening', 'night', 'late night']

flights['hour_of_day'] = flights['scheduled_departure_datetime'].dt.hour

flights['departure_window'] = pd.cut(flights['hour_of_day'], bins=bins, labels=labels)

In [37]:
flights.head()

,year,quarter,month,day_of_month,day_of_week,marketing_airline_id,flight_number_marketing_airline,operating_airline_id,tail_number,origin_airport_id,origin_city_market_id,origin,origin_state,dest_airport_id,dest_city_market_id,dest,dest_state,dep_delay,dep_del15,departure_delay_groups,taxi_out,taxi_in,arr_delay,arr_del15,arrival_delay_groups,cancelled,diverted,scheduled_elapsed_time,actual_elapsed_time,air_time,distance,distance_group,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay,div_airport_landings,code_share_flight,origin_city,destination_city,scheduled_departure_datetime,scheduled_arrival_datetime,actual_departure_datetime,actual_arrival_datetime,overnight_depature,overnight_arrival,airline_mkt,airline_ops,origin_timezone,destination_timezone,scheduled_departure_datetime_utc,actual_departure_datetime_utc,scheduled_arrival_datetime_utc,actual_arrival_datetime_utc,is_holiday,hour_of_day,departure_window,route_id
0,2023,3,8,6,7,20416,2252,20416,N978NK,12889,32211,LAS,NV,11057,31057,CLT,NC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,263.0,NaN,NaN,1916.0,8,NaN,NaN,NaN,NaN,NaN,0.0,0,Las Vegas,Charlotte,2023-08-06 23:59:00,2023-08-07 07:22:00,NaN,NaN,0,0,Spirit Air Lines,Spirit Air Lines,America/Los_Angeles,America/New_York,2023-08-07 07:00:00+00:00,NaN,2023-08-07 11:00:00+00:00,NaN,0,23,late night,3228
1,2023,3,8,7,1,20416,2252,20416,N974NK,12889,32211,LAS,NV,11057,31057,CLT,NC,76.0,1.0,5.0,19.0,9.0,69.0,1.0,4.0,0,0,263.0,256.0,228.0,1916.0,8,3.0,0.0,1.0,0.0,65.0,0.0,0,Las Vegas,Charlotte,2023-08-07 23:59:00,2023-08-08 07:22:00,2023-08-08 01:15:00,2023-08-08 08:31:00,0,0,Spirit Air Lines,Spirit Air Lines,America/Los_Angeles,America/New_York,2023-08-08 07:00:00+00:00,2023-08-08 08:15:00+00:00,2023-08-08 11:00:00+00:00,2023-08-08 12:31:00+00:00,0,23,late night,3228
2,2023,3,8,9,3,20416,2252,20416,N519NK,12889,32211,LAS,NV,11057,31057,CLT,NC,-11.0,0.0,-1.0,14.0,10.0,-13.0,0.0,-1.0,0,0,258.0,256.0,232.0,1916.0,8,NaN,NaN,NaN,NaN,NaN,0.0,0,Las Vegas,Charlotte,2023-08-09 23:10:00,2023-08-10 06:28:00,2023-08-09 22:59:00,2023-08-10 06:15:00,0,0,Spirit Air Lines,Spirit Air Lines,America/Los_Angeles,America/New_York,2023-08-10 06:00:00+00:00,2023-08-10 05:59:00+00:00,2023-08-10 10:00:00+00:00,2023-08-10 10:15:00+00:00,0,23,late night,3228
3,2023,3,8,10,4,20416,2252,20416,N532NK,12889,32211,LAS,NV,11057,31057,CLT,NC,-8.0,0.0,-1.0,12.0,7.0,-30.0,0.0,-2.0,0,0,258.0,236.0,217.0,1916.0,8,NaN,NaN,NaN,NaN,NaN,0.0,0,Las Vegas,Charlotte,2023-08-10 23:10:00,2023-08-11 06:28:00,2023-08-10 23:02:00,2023-08-11 05:58:00,0,0,Spirit Air Lines,Spirit Air Lines,America/Los_Angeles,America/New_York,2023-08-11 06:00:00+00:00,2023-08-11 06:02:00+00:00,2023-08-11 10:00:00+00:00,2023-08-11 09:58:00+00:00,0,23,late night,3228
4,2023,3,8,12,6,20416,2252,20416,N529NK,12889,32211,LAS,NV,11057,31057,CLT,NC,4.0,0.0,0.0,18.0,6.0,-4.0,0.0,-1.0,0,0,258.0,250.0,226.0,1916.0,8,NaN,NaN,NaN,NaN,NaN,0.0,0,Las Vegas,Charlotte,2023-08-12 23:10:00,2023-08-13 06:28:00,2023-08-12 23:14:00,2023-08-13 06:24:00,0,0,Spirit Air Lines,Spirit Air Lines,America/Los_Angeles,America/New_York,2023-08-13 06:00:00+00:00,2023-08-13 06:14:00+00:00,2023-08-13 10:00:00+00:00,2023-08-13 10:24:00+00:00,0,23,late night,3228


In [54]:
def calculate_flight_performance_aggregations(df, days):

    df = df.sort_values('scheduled_departure_datetime')

    grouped_df = df.groupby(['airline_mkt', 'route_id', 'departure_window'], observed=False)

    rolling_stats = grouped_df.rolling(window=days, on='scheduled_departure_datetime').agg({
    'dep_delay': ['mean', 'median', 'max'],
    'arr_delay': ['mean', 'median', 'max'],
    'cancelled': 'sum',
    'div_airport_landings': 'sum'
    }).reset_index()  # Resetting index after aggregation to avoid duplicate index issue

    # Add the number of flights by counting rows within each rolling window
    rolling_stats['n_flights'] = grouped_df.rolling(window=days, on='scheduled_departure_datetime')['dep_delay'].count().values

    # Flatten the multi-level column index for easier access
    rolling_stats.columns = [
        f'{col[0]}_{col[1]}_{days}' if isinstance(col, tuple) and col[1] != '' else col[0] for col in rolling_stats.columns
    ]
    return rolling_stats


7260941

In [55]:
rolling_stats_10D = calculate_flight_performance_aggregations(flights, '10D')
rolling_stats_10D

,airline_mkt,route_id,departure_window,scheduled_departure_datetime,dep_delay_mean_10D,dep_delay_median_10D,dep_delay_max_10D,arr_delay_mean_10D,arr_delay_median_10D,arr_delay_max_10D,cancelled_sum_10D,div_airport_landings_sum_10D,n_flights
0,Alaska Airlines Inc.,35,morning,2023-04-22 11:40:00,-1.000000,-1.0,-1.0,-10.000000,-10.0,-10.0,0.0,0.0,1.0
1,Alaska Airlines Inc.,35,morning,2023-04-23 11:40:00,-7.500000,-7.5,-1.0,-17.500000,-17.5,-10.0,0.0,0.0,2.0
2,Alaska Airlines Inc.,35,morning,2023-04-29 11:40:00,-7.333333,-7.0,-1.0,-15.666667,-12.0,-10.0,0.0,0.0,3.0
3,Alaska Airlines Inc.,35,morning,2023-04-30 11:40:00,-7.750000,-8.0,-1.0,-18.250000,-18.5,-10.0,0.0,0.0,4.0
4,Alaska Airlines Inc.,35,morning,2023-05-06 11:40:00,-5.666667,-7.0,-1.0,-20.000000,-22.0,-12.0,0.0,0.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7249990,United Air Lines Inc.,6708,early afternoon,2023-12-27 15:15:00,13.181818,-3.0,63.0,-1.181818,-16.0,62.0,0.0,0.0,11.0
7249991,United Air Lines Inc.,6708,early afternoon,2023-12-28 15:15:00,7.000000,-3.0,60.0,-4.909091,-16.0,62.0,0.0,0.0,11.0
7249992,United Air Lines Inc.,6708,early afternoon,2023-12-29 15:15:00,6.818182,-4.0,60.0,-5.181818,-16.0,62.0,0.0,0.0,11.0
7249993,United Air Lines Inc.,6708,early afternoon,2023-12-30 15:15:00,7.727273,-4.0,60.0,-4.363636,-16.0,62.0,0.0,0.0,11.0


In [56]:
def create_rolling_dfs(df, windows):

    # Define a dictionary to hold the DataFrames for each rolling window
    rolling_dfs = {}

    # Iterate through each window option and generate a DataFrame
    for window in windows:
        # Call the function and store the result in the dictionary
        rolling_dfs[f'rolling_{window}'] = calculate_flight_performance_aggregations(flights, window)

        # If you want to print or verify the DataFrame names:
        print(f"DataFrame created: rolling_{window}")

    # Now rolling_dfs['rolling_10D'], rolling_dfs['rolling_20D'], etc. hold the results for each window
    return rolling_dfs

In [57]:
rolling_dfs = create_rolling_dfs(flights, ['10D', '20D', '30D', '90D'])

DataFrame created: rolling_10D
DataFrame created: rolling_20D
DataFrame created: rolling_30D
DataFrame created: rolling_90D


In [58]:
rolling_10D = rolling_dfs['rolling_10D']
rolling_20D = rolling_dfs['rolling_20D']
rolling_30D = rolling_dfs['rolling_30D']
rolling_90D = rolling_dfs['rolling_90D']

In [59]:
rolling_10D.head()

,airline_mkt,route_id,departure_window,scheduled_departure_datetime,dep_delay_mean_10D,dep_delay_median_10D,dep_delay_max_10D,arr_delay_mean_10D,arr_delay_median_10D,arr_delay_max_10D,cancelled_sum_10D,div_airport_landings_sum_10D,n_flights
0,Alaska Airlines Inc.,35,morning,2023-04-22 11:40:00,-1.000000,-1.0,-1.0,-10.000000,-10.0,-10.0,0.0,0.0,1.0
1,Alaska Airlines Inc.,35,morning,2023-04-23 11:40:00,-7.500000,-7.5,-1.0,-17.500000,-17.5,-10.0,0.0,0.0,2.0
2,Alaska Airlines Inc.,35,morning,2023-04-29 11:40:00,-7.333333,-7.0,-1.0,-15.666667,-12.0,-10.0,0.0,0.0,3.0
3,Alaska Airlines Inc.,35,morning,2023-04-30 11:40:00,-7.750000,-8.0,-1.0,-18.250000,-18.5,-10.0,0.0,0.0,4.0
4,Alaska Airlines Inc.,35,morning,2023-05-06 11:40:00,-5.666667,-7.0,-1.0,-20.000000,-22.0,-12.0,0.0,0.0,3.0


In [60]:
flights_historical_performance = pd.merge(flights, rolling_10D, 
    on=['route_id', 'airline_mkt', 'departure_window', 'scheduled_departure_datetime'], 
    how='left', suffixes=('', '_10D'))\
    .merge(rolling_20D, 
    on=['route_id', 'airline_mkt', 'departure_window', 'scheduled_departure_datetime'], 
    how='left', suffixes=('_10D', '_20D'))\
    .merge(rolling_30D, 
    on=['route_id', 'airline_mkt', 'departure_window', 'scheduled_departure_datetime'], 
    how='left', suffixes=('_20D', '_30D'))\
    .merge(rolling_90D, 
    on=['route_id', 'airline_mkt', 'departure_window', 'scheduled_departure_datetime'], 
    how='left', suffixes=('_30D', '_90D'))

In [61]:
flights_historical_performance.head()

,year,quarter,month,day_of_month,day_of_week,marketing_airline_id,flight_number_marketing_airline,operating_airline_id,tail_number,origin_airport_id,origin_city_market_id,origin,origin_state,dest_airport_id,dest_city_market_id,dest,dest_state,dep_delay,dep_del15,departure_delay_groups,taxi_out,taxi_in,arr_delay,arr_del15,arrival_delay_groups,cancelled,diverted,scheduled_elapsed_time,actual_elapsed_time,air_time,distance,distance_group,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay,div_airport_landings,code_share_flight,origin_city,destination_city,scheduled_departure_datetime,scheduled_arrival_datetime,actual_departure_datetime,actual_arrival_datetime,overnight_depature,overnight_arrival,airline_mkt,airline_ops,origin_timezone,destination_timezone,scheduled_departure_datetime_utc,actual_departure_datetime_utc,scheduled_arrival_datetime_utc,actual_arrival_datetime_utc,is_holiday,hour_of_day,departure_window,route_id,dep_delay_mean_10D,dep_delay_median_10D,dep_delay_max_10D,arr_delay_mean_10D,arr_delay_median_10D,arr_delay_max_10D,cancelled_sum_10D,div_airport_landings_sum_10D,n_flights_10D,dep_delay_mean_20D,dep_delay_median_20D,dep_delay_max_20D,arr_delay_mean_20D,arr_delay_median_20D,arr_delay_max_20D,cancelled_sum_20D,div_airport_landings_sum_20D,n_flights_20D,dep_delay_mean_30D,dep_delay_median_30D,dep_delay_max_30D,arr_delay_mean_30D,arr_delay_median_30D,arr_delay_max_30D,cancelled_sum_30D,div_airport_landings_sum_30D,n_flights_30D,dep_delay_mean_90D,dep_delay_median_90D,dep_delay_max_90D,arr_delay_mean_90D,arr_delay_median_90D,arr_delay_max_90D,cancelled_sum_90D,div_airport_landings_sum_90D,n_flights_90D
0,2023,3,8,6,7,20416,2252,20416,N978NK,12889,32211,LAS,NV,11057,31057,CLT,NC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,263.0,NaN,NaN,1916.0,8,NaN,NaN,NaN,NaN,NaN,0.0,0,Las Vegas,Charlotte,2023-08-06 23:59:00,2023-08-07 07:22:00,NaN,NaN,0,0,Spirit Air Lines,Spirit Air Lines,America/Los_Angeles,America/New_York,2023-08-07 07:00:00+00:00,NaN,2023-08-07 11:00:00+00:00,NaN,0,23,late night,3228,5.125000,2.0,31.0,-5.000000,-6.0,16.0,3.0,0.0,8.0,11.666667,4.0,41.0,3.823529,0.0,37.0,3.0,0.0,18.0,15.518519,5.0,82.0,4.923077,0.5,62.0,4.0,0.0,27.0,12.543860,2.0,154.0,-2.142857,-7.0,124.0,4.0,0.0,57.0
1,2023,3,8,7,1,20416,2252,20416,N974NK,12889,32211,LAS,NV,11057,31057,CLT,NC,76.0,1.0,5.0,19.0,9.0,69.0,1.0,4.0,0,0,263.0,256.0,228.0,1916.0,8,3.0,0.0,1.0,0.0,65.0,0.0,0,Las Vegas,Charlotte,2023-08-07 23:59:00,2023-08-08 07:22:00,2023-08-08 01:15:00,2023-08-08 08:31:00,0,0,Spirit Air Lines,Spirit Air Lines,America/Los_Angeles,America/New_York,2023-08-08 07:00:00+00:00,2023-08-08 08:15:00+00:00,2023-08-08 11:00:00+00:00,2023-08-08 12:31:00+00:00,0,23,late night,3228,14.000000,2.0,76.0,4.714286,-6.0,69.0,3.0,0.0,8.0,13.944444,4.0,76.0,5.705882,0.0,69.0,3.0,0.0,18.0,17.740741,5.0,82.0,7.961538,5.0,69.0,4.0,0.0,27.0,13.637931,2.5,154.0,-0.894737,-7.0,124.0,4.0,0.0,58.0
2,2023,3,8,9,3,20416,2252,20416,N519NK,12889,32211,LAS,NV,11057,31057,CLT,NC,-11.0,0.0,-1.0,14.0,10.0,-13.0,0.0,-1.0,0,0,258.0,256.0,232.0,1916.0,8,NaN,NaN,NaN,NaN,NaN,0.0,0,Las Vegas,Charlotte,2023-08-09 23:10:00,2023-08-10 06:28:00,2023-08-09 22:59:00,2023-08-10 06:15:00,0,0,Spirit Air Lines,Spirit Air Lines,America/Los_Angeles,America/New_York,2023-08-10 06:00:00+00:00,2023-08-10 05:59:00+00:00,2023-08-10 10:00:00+00:00,2023-08-10 10:15:00+00:00,0,23,late night,3228,12.375000,1.5,76.0,5.000000,-6.0,69.0,3.0,0.0,8.0,13.444444,4.0,76.0,5.823529,0.0,69.0,3.0,0.0,18.0,15.185185,5.0,82.0,6.192308,0.5,69.0,4.0,0.0,27.0,13.220339,2.0,154.0,-1.103448,-7.0,124.0,4.0,0.0,59.0
3,2023,3,8,10,4,20416,2252,20416,N532NK,12889,32211,LAS,NV,11057,31057,CLT,NC,-8.0,0.0,-1.0,12.0,7.0,-30.0,0.0,-2.0,0,0,258.0,236.0,217.0,1916.0,8,NaN,NaN,NaN,NaN,NaN,0.0,0,Las Vegas,Charlotte,2023-08-10 23:10:00,2023-08-11 06:28:00,2023-08-10 23:02:00,2023-08-11 05:58:00,0,0,Spirit Air Lines,Spirit Air Lines,America/Los_Angeles,America/New_York,2023-08-11 06:00:00+00:00,2023-08-11 06:02:00+0

In [62]:
flights_historical_performance.shape

(7265501, 95)

In [67]:
flights_historical_performance.to_csv(DATA_PATH + '/interim/flights_historical_performance.csv', index=False)


KeyboardInterrupt: 

In [68]:
rolling_10D.duplicated(subset=['route_id', 'airline_mkt', 'departure_window', 'scheduled_departure_datetime']).sum()


152

In [73]:
rolling_10D[rolling_10D.duplicated(subset=['route_id', 'airline_mkt', 'departure_window', 'scheduled_departure_datetime'])].head()


,airline_mkt,route_id,departure_window,scheduled_departure_datetime,dep_delay_mean_10D,dep_delay_median_10D,dep_delay_max_10D,arr_delay_mean_10D,arr_delay_median_10D,arr_delay_max_10D,cancelled_sum_10D,div_airport_landings_sum_10D,n_flights
1717937,American Airlines Inc.,4372,midday,2023-06-15 13:00:00,16.900000,-11.0,109.0,7.300000,-19.5,111.0,1.0,0.0,10.0
1911622,American Airlines Inc.,4942,morning,2023-06-15 10:41:00,11.333333,-6.0,133.0,6.111111,-8.0,113.0,1.0,0.0,18.0
2280806,Delta Air Lines Inc.,165,morning,2023-07-13 08:37:00,14.437500,-1.0,196.0,13.000000,-3.0,189.0,1.0,0.0,16.0
2280809,Delta Air Lines Inc.,165,morning,2023-07-14 08:37:00,13.647059,-1.0,196.0,12.176471,-3.0,189.0,2.0,0.0,17.0
2280844,Delta Air Lines Inc.,165,morning,2023-08-01 09:53:00,-0.315789,-2.0,35.0,-3.526316,-6.0,25.0,2.0,0.0,19.0


In [69]:
rolling_20D.duplicated(subset=['route_id', 'airline_mkt', 'departure_window', 'scheduled_departure_datetime']).sum()


152

In [70]:
rolling_30D.duplicated(subset=['route_id', 'airline_mkt', 'departure_window', 'scheduled_departure_datetime']).sum()


152

In [71]:
rolling_90D.duplicated(subset=['route_id', 'airline_mkt', 'departure_window', 'scheduled_departure_datetime']).sum()


152

In [74]:
flights.duplicated(subset=['route_id', 'airline_mkt', 'departure_window', 'scheduled_departure_datetime']).sum()

152

In [79]:
# Identify duplicated rows based on the key columns
duplicate_mask = flights.duplicated(subset=['route_id', 'airline_mkt', 'departure_window', 'scheduled_departure_datetime'], keep=False)

# Filter out the duplicated rows to inspect them
duplicate_rows = flights[duplicate_mask]

# Display a sample of the duplicates to investigate
duplicate_rows.sort_values(['scheduled_departure_datetime', 'airline_mkt']).head(10)


,year,quarter,month,day_of_month,day_of_week,marketing_airline_id,flight_number_marketing_airline,operating_airline_id,tail_number,origin_airport_id,origin_city_market_id,origin,origin_state,dest_airport_id,dest_city_market_id,dest,dest_state,dep_delay,dep_del15,departure_delay_groups,taxi_out,taxi_in,arr_delay,arr_del15,arrival_delay_groups,cancelled,diverted,scheduled_elapsed_time,actual_elapsed_time,air_time,distance,distance_group,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay,div_airport_landings,code_share_flight,origin_city,destination_city,scheduled_departure_datetime,scheduled_arrival_datetime,actual_departure_datetime,actual_arrival_datetime,overnight_depature,overnight_arrival,airline_mkt,airline_ops,origin_timezone,destination_timezone,scheduled_departure_datetime_utc,actual_departure_datetime_utc,scheduled_arrival_datetime_utc,actual_arrival_datetime_utc,is_holiday,hour_of_day,departure_window,route_id
7194903,2023,1,3,5,7,20436,106,20436,N610FR,13204,31454,MCO,FL,14843,34819,SJU,PR,1.0,0.0,0.0,20.0,7.0,6.0,0.0,0.0,0,0,168.0,173.0,146.0,1189.0,5,NaN,NaN,NaN,NaN,NaN,0.0,0,Orlando,San Juan,2023-03-05 05:45:00,2023-03-05 09:33:00,2023-03-05 05:46:00,2023-03-05 09:39:00,1,0,Frontier Airlines Inc.,Frontier Airlines Inc.,America/New_York,America/Puerto_Rico,2023-03-05 11:00:00+00:00,2023-03-05 10:46:00+00:00,2023-03-05 14:00:00+00:00,2023-03-05 13:39:00+00:00,0,5,early morning,3781
7194907,2023,1,3,5,7,20436,110,20436,N714FR,13204,31454,MCO,FL,14843,34819,SJU,PR,-5.0,0.0,-1.0,14.0,8.0,-14.0,0.0,-1.0,0,0,176.0,167.0,145.0,1189.0,5,NaN,NaN,NaN,NaN,NaN,0.0,0,Orlando,San Juan,2023-03-05 05:45:00,2023-03-05 09:41:00,2023-03-05 05:40:00,2023-03-05 09:27:00,1,0,Frontier Airlines Inc.,Frontier Airlines Inc.,America/New_York,America/Puerto_Rico,2023-03-05 11:00:00+00:00,2023-03-05 10:40:00+00:00,2023-03-05 14:00:00+00:00,2023-03-05 13:27:00+00:00,0,5,early morning,3781
7111005,2023,1,3,14,2,19790,315,19790,NaN,10397,30397,ATL,GA,14893,33192,SMF,CA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,320.0,NaN,NaN,2092.0,9,NaN,NaN,NaN,NaN,NaN,0.0,0,Atlanta,Sacramento,2023-03-14 09:10:00,2023-03-14 11:30:00,NaN,NaN,0,0,Delta Air Lines Inc.,Delta Air Lines Inc.,America/New_York,America/Los_Angeles,2023-03-14 13:00:00+00:00,NaN,2023-03-14 18:00:00+00:00,NaN,0,9,morning,288
7115454,2023,1,3,14,2,19790,454,19790,N915DU,10397,30397,ATL,GA,14893,33192,SMF,CA,-2.0,0.0,-1.0,13.0,4.0,-10.0,0.0,-1.0,0,0,320.0,312.0,295.0,2092.0,9,NaN,NaN,NaN,NaN,NaN,0.0,0,Atlanta,Sacramento,2023-03-14 09:10:00,2023-03-14 11:30:00,2023-03-14 09:08:00,2023-03-14 11:20:00,0,0,Delta Air Lines Inc.,Delta Air Lines Inc.,America/New_York,America/Los_Angeles,2023-03-14 13:00:00+00:00,2023-03-14 13:08:00+00:00,2023-03-14 18:00:00+00:00,2023-03-14 18:20:00+00:00,0,9,morning,288
7111006,2023,1,3,14,2,19790,315,19790,NaN,14893,33192,SMF,CA,10397,30397,ATL,GA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,263.0,NaN,NaN,2092.0,9,NaN,NaN,NaN,NaN,NaN,0.0,0,Sacramento,Atlanta,2023-03-14 12:40:00,2023-03-14 20:03:00,NaN,NaN,0,0,Delta Air Lines Inc.,Delta Air Lines Inc.,America/Los_Angeles,America/New_York,2023-03-14 20:00:00+00:00,NaN,2023-03-15 00:00:00+00:00,NaN,0,12,midday,6191
7115455,2023,1,3,14,2,19790,454,19790,N915DU,14893,33192,SMF,CA,10397,30397,ATL,GA,6.0,0.0,0.0,12.0,7.0,-10.0,0.0,-1.0,0,0,263.0,247.0,228.0,2092.0,9,NaN,NaN,NaN,NaN,NaN,0.0,0,Sacramento,Atlanta,2023-03-14 12:40:00,2023-03-14 20:03:00,2023-03-14 12:46:00,2023-03-14 19:53:00,0,0,Delta Air Lines Inc.,Delta Air Lines Inc.,America/Los_Angeles,America/New_York,2023-03-14 20:00:00+00:00,2023-03-14 19:46:00+00:00,2023-03-15 00:00:00+00:00,2023-03-14 23:53:00+00:00,0,12,midday,6191
3829568,2023,2,4,13,4,20409,1903,20409,N2016J,12478,31703,JFK,NY,14843,34819,SJU,PR,158.0,1.0,10.0,17.0,5.0,130.0,1.0,8.0,0,0,239.0,211.0,189.0,1598.0,7,8.0,0.0,0.0,0.0,122.0,0.0,0,New York,San Juan,2023-04-13 21:59:00,2023-04-14 01:58:00,2023-04-14 00:37:00,2023-04-14 04:08:00,0,1,JetBlue Airways,JetBlue Airways,America/New_York,